In [ ]:
# Below we have sections of short driver scripts that run different parts of the heatmap code
# Not all functions in the 5 functiond definition .py's are called yet,
# but the ones that are called so far compile below without error

In [ ]:
import importlib
import setup_nx # your own module, setup.nx.py
import numpy as np
import math as m
import statistics as st
import cmath
import matplotlib.pyplot as plt 
import itertools
import random
from operator import add
importlib.reload(setup_nx)
from setup_nx import *
from graphviz import Source, render

import datetime
import time

import my_feeder_funcs as ff
import my_impedance_funcs as imp
import my_configVis_funcs as vis
import my_detControlMatExistence_funcs as ctrl
import my_detLznRange_funcs as lzn
import my_heatmapSetup_funcs as hm


In [ ]:
# # List functions in each helper .py
# help(ff)
# print("----------------------------------------------------------")
# help(imp)
# print("----------------------------------------------------------")
# help(vis)
# print("----------------------------------------------------------")
# help(lzn)
# print("----------------------------------------------------------")
# help(hm)
# print("----------------------------------------------------------")
# help(ctrl)

In [ ]:
# [Essential] specify input feeder data

#-----------------------------------------------------------------
'specifying file paths'
# Enter the path/name of the model's excel file and import
    # All GridBright load files should be in the following folder
    #loadfolder = "/Users/jasperpakshong/Documents/Berkeley/ENERGISE/IEEE13/"
    #loadpath = loadfolder + "IEEE13testload_w_extreme_act.xlsx"

    # filepath = "IEEE13/"
    # modelpath = filepath + "001 phasor08_IEEE13_OPAL.xls"
    # loadfolder = "IEEE13/"
    # loadpath = loadfolder + "001_phasor08_IEEE13_norm03_HIL_7_1.xlsx"

    #filepath = "AL0001/"
    #modelpath = filepath + "AL0001_OPAL_working.xls"
    #loadfolder = "AL0001/"
    #loadpath = loadfolder + "AL0001_tvload_afternoon1h.csv"
      
    #filepath = "13NF_balanced/"
    #modelpath = filepath + "016 GB_IEEE13_balance_all_ver2.xls"
    #loadfolder = "13NF_balanced/"
    #loadpath = loadfolder + "016 GB_IEEE13_balance all ver2_time_sigBuilder_secondWise_norm03.csv"

filepath = "123NF/"
modelpath = filepath + "004_GB_IEEE123_OPAL.xls"
loadfolder = "123NF/"
loadpath = loadfolder + "004_GB_IEEE123_time_sigBuilder_secondwise_norm1.csv"
headerpath = '??13NF_balanced/001_phasor08_IEEE13_time_header.csv'
load_data = '??13NF_balanced/001_phasor08_IEEE13_time_PVpen50_cloud_minutewise (1).csv'


#==========================================================================================================

'specifying file name'
#file_name = string specifying name of dot file created when make_graph() is called
file_name = '123NF_test.dot'


#==========================================================================================================

'Specify substation kV, kVA bases, name, and the number of timesteps in the load data'
Vbase_ll = 4160
Vbase = Vbase_ll / np.sqrt(3)
Sbase = 5000/3
substation_name = 'bus_150'
timesteps = 1

'DO NOT NEED TO EDIT THIS CELL BEFORE RUNNING'

ts = time.time()
print()
print(datetime.datetime.fromtimestamp(ts))

plot = 0 #turn plot on/off

depths = {}
leaves = []

In [ ]:
# [ESSENTIAL] create feeder obj
fin_feeder = ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
print("Finished initializing feeder")
ff.make_graph(fin_feeder, file_name)
node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix
R,X=hm.createRXmatrices_3ph(fin_feeder, node_index_map,depths)

print(depths) # should be populated

count = 0 # print list of buses in network
for i in fin_feeder.network:    
    print(i) 
    count += 1
    if count >= 10:
        break
    
Source.from_file(file_name)
#^ need this to plot feeder

In [ ]:
import csv 
graph = fin_feeder.network
#print(graph.nodes)

# write busnames into a csv
# with open("123NF_busList.csv", 'w', newline='') as csvfile:
#     spamwriter = csv.writer(csvfile, delimiter='-',
#                             quotechar='|', quoting=csv.QUOTE_MINIMAL)
#     spamwriter.writerows(graph.nodes)

In [ ]:
# [Optional] run impedance-related functions
slack_bus = None
for bus_name, depth in depths.items():
    if depth == 0:
        slack_bus = bus_name
        break
        
# -------------------- now we call functions: ---------------------------------------------
#print(depths)
# modify node names when change feeders
#plot_histogram_RX_ratios(fin_feeder, leaves_only = True)
print('Z between buses:')
print(np.around(imp.get_total_impedance_between_two_buses(fin_feeder, 'bus_37', 'bus_15',depths),2))
print('\nBus 49 Z to substation:')
print(np.around(imp.get_total_impedance_from_substation(fin_feeder, 'bus_49',depths),2))
print('\nR/X ratio of bus 49 to substation:')
print(imp.get_RX_ratio_tosubst(fin_feeder,'bus_49',depths))
# should check on how to format the printing do that it's to like 2 decimal places

print('R/X Ratios:')
print('\n67 to 79: ')
print(imp.get_RX_ratio_between_two_buses(fin_feeder, 'bus_67', 'bus_79',depths))
print('\n67 to 96: ')
print(imp.get_RX_ratio_between_two_buses(fin_feeder, 'bus_67', 'bus_96',depths))
print('\n67 to 104: ')
print(imp.get_RX_ratio_between_two_buses(fin_feeder, 'bus_67', 'bus_104',depths))
print('\n67 to 114: ')
print(imp.get_RX_ratio_between_two_buses(fin_feeder, 'bus_67', 'bus_114',depths))
print('\n57 to 66: ')
print(imp.get_RX_ratio_between_two_buses(fin_feeder, 'bus_57', 'bus_66',depths))
print('\n18 to 251: ')
print(imp.get_RX_ratio_between_two_buses(fin_feeder, 'bus_18', 'bus_251',depths))
print('\n35 to 151: ')
print(imp.get_RX_ratio_between_two_buses(fin_feeder, 'bus_35', 'bus_151',depths))

In [ ]:
# Print 3-phase node_index table
my_buses=list(node_index_map.keys())
my_list=list(node_index_map.values())
list1 = [(i+1) * 3-2 for i in my_list]
list2 = [(i+1) * 3-1 for i in my_list]
list3 = [(i+1) * 3-0 for i in my_list]
for my_buses,list1, list2,list3 in zip(my_buses,list1,list2,list3):
    print(my_buses,list1, list2,list3)
#Table format: [bus_name, phaseA idx, phaseB idx, phaseC idx]
# indices are shifted by 1 to be for MATLAB (starts at 1 not 0)

In [ ]:
# R is of size 387, which is 129*3, so set n=129
n=129
A, B = hm.setupStateSpace(n,fin_feeder, node_index_map,depths)
print('R = '+str(R))
print('X = '+str(X))
print('A = '+str(A))
print('B = '+str(B))
print(A.shape)
print(B.shape)

# write A and B matrices to csv
#np.savetxt("123NF_Amat.csv", A, delimiter=",")
#np.savetxt("123NF_Bmat.csv", B, delimiter=",")

# Save R and X matrices to csv to import into matlab
# np.savetxt reference: https://thispointer.com/how-to-save-numpy-array-to-a-csv-file-using-numpy-savetxt-in-python/
#np.savetxt('Rmat_123NF.csv', R, delimiter=',')
#np.savetxt('Xmat_123NF.csv', X, delimiter=',')

In [ ]:
# Create figure 2
ratios = vis.phaseCouplingPerNode(fin_feeder,depths)
vis.createColorMap(fin_feeder, ratios, 'fig2_colorMap_123NF')
Source.from_file('fig2_colorMap_123NF') # display graph in notebook
ff.clear_graph(fin_feeder)


In [ ]:
# detLznRange(feeder, Vbase_ll, Sbase, z12, act_locs)
#^ calls computePQsweep, makePVcurve, makeQVcurve, computeLznItvl
#loadfolder = "13NF_balanced/"
#load_data = '001_phasor08_IEEE13_time_PVpen50_cloud_minutewise (1).csv'

#lzn.computePQsweep_timesteps(fin_feeder, loadfolder+load_data)


In [ ]:
# commented out because takes LONG time to evaluate co-located at every node of 123NF
#-----------------------------------------------

# Create Fig 1: # start with just binary heat map across all nodes
# based on heat map colors, maybe try: branches color-coded and marked feasible (green outline) or infeasible (red outline) 

# # find good colocated?
# all_act_locs = []
# perf_nodes = []
# # empty act locs asks heatMapProcess to create heatmap for empty network, i.e. feas of placing 1 co-located actuator at each loc
# feas_configs, heatMapNames=hm.find_good_colocated(fin_feeder, [], node_index_map,substation_name,depths, file_name)
# print('----------- Feas configs are: ---------')
# print(feas_configs)
# Source.from_file(heatMapNames[0]) # display graph in notebook

In [ ]:
# ----- Run computeFParamSpace ---------
all_act_locs = ['bus_25','bus_39','bus_56','bus_108'] # 49, then 76, then 65
perf_nodes = ['bus_25','bus_39','bus_56','bus_108']
Fq_ub,Fp_ub=ctrl.computeFParamSpace_v2(fin_feeder, all_act_locs, perf_nodes,R,X,depths,node_index_map)
# this function prints Zgood (from R and X matrices) and Z_toSubst, where Z_toSubst is correct but doesnt match Zgood. Investigate
print('(Fp_ub,Fq_ub)=(',Fp_ub,',',Fq_ub,')')


vis.markActuatorConfig(all_act_locs, fin_feeder, file_name)

In [ ]:
# -------Create Fig 4 ----------
# ----------- Demo of RHP on FEAS config --------------

# binary heat map that shows good locations to place a new actuator when
# a) there are 0 existing actuators, b) there is 1 existing actuator, and c) there are two existing actuators.
# actuator list = complex config 1, i.e. at each step we choose one of the good places and buildup to complex config 1

# #complex config 1 (feas)
#all_act_locs = ['bus_49','bus_76','bus_65'] # 49, then 76, then 65
#perf_nodes = ['bus_49','bus_76','bus_65']
# 16336 seconds 

#complex config 2 (feas)
# all_act_locs = ['bus_25','bus_39','bus_56','bus_108'] # 49, then 76, then 65
# perf_nodes = ['bus_25','bus_39','bus_56','bus_108']
# 30206 seconds

# #complex config 3 (feas)
# all_act_locs = ['bus_105','bus_112','bus_100','bus_91'] # 49, then 76, then 65
# perf_nodes = ['bus_105','bus_112','bus_100','bus_91']
# # time taken = 21674 seconds

# #complex config 4, many act 1 perf
#all_act_locs = ['bus_42','bus_42','bus_42','bus_42'] # 49, then 76, then 65
#perf_nodes = ['bus_151','bus_46','bus_251','bus_8']
# # time taken = XXX seconds

      
t = time.time()
# complex config 2
all_act_locs = ['bus_25','bus_39','bus_56','bus_108'] # 49, then 76, then 65
perf_nodes = ['bus_25','bus_39','bus_56','bus_108']
lst_feas_configs, lzn_error_run_sum,heatMapNames=hm.runHeatMapProcess(fin_feeder, all_act_locs, perf_nodes, node_index_map,substation_name,depths,file_name)
elapsed = time.time() - t
print('Time elapsed=',elapsed)
      
    

In [ ]:
# all_act_locs = ['bus_49','bus_76','bus_152'] # 49, then 76, then 65
# perf_nodes = ['bus_49','bus_76','bus_65']
# feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name)

In [ ]:
# # Place maximum number of colocated act/perf nodes on network and display configuration
# max_act_config = hm.place_max_colocated_acts(fin_feeder, file_name, node_index_map, depths, substation_name)
# Source.from_file('actConfig_max_coloc_act_config')


In [ ]:
# if file_name=='13NF_test.dot':
#     substIdx=[6, 7] # substation index
# elif file_name=='123NF_test.dot':
#     substIdx=[22, 24]
# elif file_name=='PL0001_test.dot':
#     substIdx=[22, 24] # dunno yet
# graphNodes_nosub=np.delete(graph.nodes,substIdx) # vector of strings
# idx=range(len(graphNodes_nosub))
# #print(idx[3:]) # first 3 elements
# #print(idx[:3]) # 3th to end

# print(graphNodes_nosub[51:79+1])
# all_act_locs=graphNodes_nosub[51:79+1] # all non-substation nodes
# print(all_act_locs)
# n=len(all_act_locs)
# perf_nodes=['bus_46'] * n
# feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name)
# vis.markActuatorConfig(all_act_locs, fin_feeder, file_name)

# print(graphNodes_nosub[45:79+1])
# all_act_locs=graphNodes_nosub[45:79+1] # all non-substation nodes
# perf_nodes=['bus_46'] * n
# feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name)


In [ ]:
# # Find index in graphNodes_nosub array of busnames strings
# if file_name=='13NF_test.dot':
#     substIdx=[6, 7] # substation index
# elif file_name=='123NF_test.dot':
#     substIdx=[22, 24]
# elif file_name=='PL0001_test.dot':
#     substIdx=[22, 24] # dunno yet
# graphNodes_nosub=np.delete(graph.nodes,substIdx) # vector of strings 
# bus2find='bus_35'
# idx = [idx for idx, element in enumerate(graphNodes_nosub) if element==bus2find]
# print(idx)



In [ ]:
Source.from_file(heatMapNames[0]) # display graph in notebook

In [ ]:
Source.from_file(heatMapNames[1]) # display graph in notebook

In [ ]:
Source.from_file(heatMapNames[2]) # display graph in notebook